In [1]:
import os
import subprocess

import numpy as np
import matplotlib.pyplot as plt
from corner import corner
import seaborn as sns
import emcee
import dill as pickle

import model

In [2]:
b = emcee.backends.HDFBackend('emulators/backends/good_eps_1.0000e-02_ntrain_700_fat_0.3.h5')

with open('emulators/emu_good_eps_1.0000e-02_ntrain_700_fat_0.3.pkl', 'rb') as f:
    emu = pickle.load(f)

In [3]:
from bayes import Model3

In [4]:
m = Model3(emu)

In [5]:
flat_chain = b.get_chain(flat=True)

In [6]:
mu, cov = m.gp_predict(flat_chain[0, :16])

In [7]:
mu.shape, cov.shape

((573,), (573, 573))

In [8]:
-0.5*np.linalg.slogdet(np.diag(model.dy**2) + cov)[1]

2389.4080175381046

In [9]:
np.linalg.slogdet(cov)[1]

-21698.176306600253

In [10]:
m.likelihood_normalization(cov)

10848.169214766922

In [11]:
m.ln_likelihood(flat_chain[0])

1146.499928656118

In [12]:
m.chi_squared(model.y, mu, cov)

-1.045929061707564e+43

In [13]:
mu @ cov @ mu, mu @ cov @ mu[:, np.newaxis]

(1431171.8968181643, array([1431171.89681816]))

In [2]:
f = np.load('datfiles/posterior_samples.npy')
w = np.load('datfiles/posterior_chain.npy')

f.shape, w.shape

((1000, 573), (1000, 32))

In [3]:
def pop(array, i):
    x = array[i]
    array = np.delete(array, i)
    return x, array

In [4]:
x = np.arange(10)
x

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [5]:
pop(x, 2)

(2, array([0, 1, 3, 4, 5, 6, 7, 8, 9]))

In [6]:
X = np.random.rand(3, 4)
X

array([[0.85830398, 0.9024815 , 0.79444778, 0.14809394],
       [0.70110264, 0.91301548, 0.05566102, 0.74891158],
       [0.37174993, 0.68970549, 0.45068396, 0.75856722]])

In [7]:
pop(X, 1)

(array([0.70110264, 0.91301548, 0.05566102, 0.74891158]),
 array([0.85830398, 0.79444778, 0.14809394, 0.70110264, 0.91301548,
        0.05566102, 0.74891158, 0.37174993, 0.68970549, 0.45068396,
        0.75856722]))

In [8]:
ntrain = 50
f = data[:ntrain, :].T
theta = points[:ntrain, :16]
x = model.x.copy()[:, np.newaxis]

f.shape, theta.shape, x.shape

NameError: name 'data' is not defined

In [39]:
args = {'warnings' : True}
emu = emulator(x=x, theta=theta, f=f, method='PCGPwM', args=args)

PCGPwM considering  28 PCs


In [40]:
z = (f - np.mean(f, axis=1)[:, np.newaxis]) / np.std(f, axis=1)[:, np.newaxis]
z.shape

(573, 50)

In [41]:
evals, evecs = np.linalg.eig(z @ z.T)

In [42]:
ii = np.argsort(evals)
evals = evals[ii]
evecs = evecs[:, ii]

In [43]:
evals[-1], evecs[:, -1]

((19040.03975761858+0j),
 array([ 0.00946321+0.j,  0.00942363+0.j,  0.00855174+0.j,  0.00733041+0.j,
         0.0039244 +0.j,  0.00125663+0.j, -0.00807809+0.j, -0.00807137+0.j,
         0.00888423+0.j,  0.00860497+0.j,  0.00855447+0.j, -0.00578613+0.j,
        -0.01005356+0.j, -0.01005356+0.j, -0.01179858+0.j, -0.01215192+0.j,
        -0.01252565+0.j,  0.00942026+0.j,  0.00956364+0.j,  0.009102  +0.j,
         0.00700669+0.j,  0.00482764+0.j,  0.00482827+0.j,  0.00078057+0.j,
        -0.00026063+0.j, -0.00133235+0.j, -0.00249444+0.j, -0.00374554+0.j,
        -0.00543692+0.j, -0.00627033+0.j, -0.00670892+0.j, -0.00855329+0.j,
        -0.00975717+0.j, -0.01050721+0.j, -0.00656175+0.j, -0.00511045+0.j,
        -0.00375901+0.j, -0.00292535+0.j, -0.00164665+0.j, -0.00094791+0.j,
         0.00043854+0.j,  0.00044154+0.j, -0.0052036 +0.j, -0.00393361+0.j,
        -0.00252848+0.j, -0.0002824 +0.j, -0.008603  +0.j, -0.00855647+0.j,
        -0.0084707 +0.j, -0.00836362+0.j, -0.00823366+0.j, -0.0

In [44]:
z.T.shape

(50, 573)

In [45]:
U, S, vh = np.linalg.svd(z.T, full_matrices=False)

In [46]:
U.shape, S.shape, vh.shape

((50, 50), (50,), (50, 573))

In [47]:
S**2

array([1.90400398e+04, 4.08936139e+03, 1.85511336e+03, 1.13882508e+03,
       6.06213016e+02, 5.71625992e+02, 4.01711985e+02, 2.87821836e+02,
       1.91665391e+02, 1.38754786e+02, 1.24886964e+02, 7.44317819e+01,
       4.26558346e+01, 3.47885665e+01, 2.79692419e+01, 1.28992989e+01,
       4.10724687e+00, 1.79562444e+00, 1.37529675e+00, 1.13311952e+00,
       6.54212735e-01, 3.04029092e-01, 2.35103406e-01, 2.18302119e-01,
       1.77968045e-01, 1.44723425e-01, 1.35286105e-01, 1.09654610e-01,
       9.81420300e-02, 9.24727908e-02, 7.20429026e-02, 6.79967389e-02,
       5.87081779e-02, 5.42224465e-02, 5.36769168e-02, 4.53053361e-02,
       4.21208379e-02, 3.85345365e-02, 3.54416555e-02, 3.25543760e-02,
       2.67418462e-02, 2.37722411e-02, 2.26973426e-02, 1.80137876e-02,
       1.69782649e-02, 1.40684314e-02, 1.36461200e-02, 1.02497104e-02,
       7.76833839e-03, 1.16212123e-21])